# Hey Bert!

This is my attempt to teach myself how BERT works.

## Imports

In [1]:
# Install external module(s)

import os
import string

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import transformers as trf
import yaml


if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

EPOCHS = 3
LEARNING_RATE = 0.01

/usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Load Data

In [3]:
# Load training and testing data
DATA_DIR = 'kaggle/input/google-quest-challenge'
print('Loading data...  ', end='')
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
sample_submission_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
print('Done!')

# Sort data into separate groups
print('Sorting data...  ', end='')
train_titles = train_df['question_title'].values
train_bodies = train_df['question_body'].values
train_answers = train_df['answer'].values

test_labels = sample_submission_df.columns
test_qa_ids = test_df['qa_id'].values
test_titles = test_df['question_title'].values
test_bodies = test_df['question_body'].values
test_answers = test_df['answer'].values

targets = torch.tensor(train_df[train_df.columns[11:]].values, dtype=torch.float, device=DEVICE)
print('Done!')

Loading data...  Done!
Sorting data...  Done!


## Process Data

In [4]:
tokenizer = trf.BertTokenizer.from_pretrained('bert-base-uncased')
help(tokenizer)


Help on BertTokenizer in module transformers.tokenization_bert object:

class BertTokenizer(transformers.tokenization_utils.PreTrainedTokenizer)
 |  BertTokenizer(vocab_file, do_lower_case=True, do_basic_tokenize=True, never_split=None, unk_token='[UNK]', sep_token='[SEP]', pad_token='[PAD]', cls_token='[CLS]', mask_token='[MASK]', tokenize_chinese_chars=True, **kwargs)
 |  
 |  Constructs a BertTokenizer.
 |  :class:`~transformers.BertTokenizer` runs end-to-end tokenization: punctuation splitting + wordpiece
 |  
 |  Args:
 |      vocab_file: Path to a one-wordpiece-per-line vocabulary file
 |      do_lower_case: Whether to lower case the input. Only has an effect when do_basic_tokenize=True
 |      do_basic_tokenize: Whether to do basic tokenization before wordpiece.
 |      max_len: An artificial maximum length to truncate tokenized sequences to; Effective maximum length is always the
 |          minimum of this value (if specified) and the underlying BERT model's sequence length.


In [ ]:
def clean(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def get_unique_words(text):
    text = clean(text).lower()
    return set(text.split())
    
def tokenize(text, vocab):
    text = clean(text).lower()
    tokens = torch.tensor([vocab[w] for w in text.split()], dtype=torch.long, device=DEVICE)
    tokens = torch.tensor([vocab['']], device=DEVICE) if tokens.size(0) == 0 else tokens
    return tokens
    
# Create vocabulary of all words
print('Extracting vocabulary words...  ', end='')
vocabulary = set()
for text in train_titles:
    vocabulary = vocabulary.union(get_unique_words(text))
for text in train_bodies:
    vocabulary = vocabulary.union(get_unique_words(text))
for text in train_answers:
    vocabulary = vocabulary.union(get_unique_words(text))
for text in test_titles:
    vocabulary = vocabulary.union(get_unique_words(text))
for text in test_bodies:
    vocabulary = vocabulary.union(get_unique_words(text))
for text in test_answers:
    vocabulary = vocabulary.union(get_unique_words(text)) 
print(f'Done!')
    
vocabulary = {word:token for token, word in enumerate(vocabulary, start=1)}
vocabulary[''] = 0

# Tokenize data
print('Tokenizing data...  ', end='')
train_title_tokens = []
train_body_tokens = []
train_answer_tokens = []
test_title_tokens = []
test_body_tokens = []
test_answer_tokens = []
for title, body, answer in zip(train_titles, train_bodies, train_answers):
    train_title_tokens.append(tokenize(title, vocabulary))
    train_body_tokens.append(tokenize(body, vocabulary))
    train_answer_tokens.append(tokenize(answer, vocabulary))
for title, body, answer in zip(test_titles, test_bodies, test_answers):
    test_title_tokens.append(tokenize(title, vocabulary))
    test_body_tokens.append(tokenize(body, vocabulary))
    test_answer_tokens.append(tokenize(answer, vocabulary))
print('Done!')

# Get max lengths
print('Calculating maximum sequence lengths...  ', end='')
max_title_length = max(max([t.size(0) for t in train_title_tokens]), max([t.size(0) for t in test_title_tokens]))
max_body_length = max(max([t.size(0) for t in train_body_tokens]), max([t.size(0) for t in test_body_tokens]))
max_answer_length = max(max([t.size(0) for t in train_answer_tokens]), max([t.size(0) for t in test_answer_tokens]))
print('Done!')

## The Neural Network

In [ ]:
print('Setting up neural networks...  ', end='')

class Encoder(nn.Module):
    def __init__(self, vocab_size, max_length, embedding_dim=512):
        super(Encoder, self).__init__()
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.embedding_dim = embedding_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim).to(device=DEVICE)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8).to(device=DEVICE)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=6).to(device=DEVICE)
        
    def forward(self, x):
        x = x.unsqueeze(0)
        embedded = self.embedding(x)
        y = self.transformer_encoder(embedded)
        padding = torch.zeros(1, self.max_length - y.size(1), y.size(2), device=DEVICE)
        y = torch.cat((y, padding), dim=1).unsqueeze(0)
        return y

class Network(nn.Module):
    def __init__(self, title_encoder, body_encoder, answer_encoder, kernel_size=7, pad_size=3):
        super(Network, self).__init__()
        self.title_encoder = title_encoder
        self.body_encoder = body_encoder
        self.answer_encoder = answer_encoder
        self.kernel_size = kernel_size
        self.pad_size = pad_size
        
        # CNNs
        self.conv1 = nn.Conv2d(1, 5, kernel_size=self.kernel_size).to(device=DEVICE)
        self.conv2 = nn.Conv2d(5, 15, kernel_size=self.kernel_size).to(device=DEVICE)
        self.conv3 = nn.Conv2d(15, 30, kernel_size=self.kernel_size).to(device=DEVICE)
        self.conv4 = nn.Conv2d(30, 50, kernel_size=self.kernel_size).to(device=DEVICE)
        
        # Find the size to flatten CNN output
        x = torch.rand(1, 1, title_encoder.max_length + body_encoder.max_length + answer_encoder.max_length, 512, device=DEVICE)
        y = F.relu(F.max_pool2d(self.conv1(x), self.pad_size))
        y = F.relu(F.max_pool2d(self.conv2(y), self.pad_size))
        y = F.relu(F.max_pool2d(self.conv3(y), self.pad_size))
        y = F.relu(F.max_pool2d(self.conv4(y), self.pad_size))
        self.cnn_flatten_size = y.size(1) * y.size(2) * y.size(3)
        
        # Linear layers
        self.lin1 = nn.Linear(self.cnn_flatten_size, 1650).to(device=DEVICE)
        self.lin2 = nn.Linear(1650, 412).to(device=DEVICE)
        self.lin3 = nn.Linear(412, 30).to(device=DEVICE)
    
    def forward(self, title, body, answer):
        title_y = self.title_encoder(title)
        body_y = self.body_encoder(body)
        answer_y = self.answer_encoder(answer)
        y = torch.cat((title_y, body_y, answer_y), dim=2)
        y = F.relu(F.max_pool2d(self.conv1(y), self.pad_size))
        y = F.relu(F.max_pool2d(self.conv2(y), self.pad_size))
        y = F.relu(F.max_pool2d(self.conv3(y), self.pad_size))
        y = F.relu(F.max_pool2d(self.conv4(y), self.pad_size))
        y = y.view(-1, self.cnn_flatten_size)
        y = F.relu(self.lin1(y))
        y = F.relu(self.lin2(y))
        y = torch.sigmoid(self.lin3(y))
        return y.squeeze()
    
    
title_encoder = Encoder(len(vocabulary), max_title_length)
body_encoder = Encoder(len(vocabulary), max_body_length)
answer_encoder = Encoder(len(vocabulary), max_answer_length)
network = Network(title_encoder, body_encoder, answer_encoder)

optimizer = optim.SGD(network.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()
print('Done!')

## Training

In [ ]:
print('\n--- Begin Training ---')
train_data = list(enumerate(zip(train_title_tokens, train_body_tokens, train_answer_tokens, targets)))
total = len(train_data)
network.train()
for epoch in range(1, EPOCHS + 1):
    total_loss = 0
    for idx, (title, body, answer, target) in train_data:
        optimizer.zero_grad()
        output = network(title, body, answer)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        print(f'Epoch: {epoch:{len(str(EPOCHS))}d}/{EPOCHS}  Loss: {total_loss / (idx + 1):.4f}  Item: {idx + 1:{len(str(total))}d}/{total}', end='\r')
    print()

## Testing

In [ ]:
network.train(False)
test_data = list(enumerate(zip(test_qa_ids, test_title_tokens, test_body_tokens, test_answer_tokens)))
total = len(test_data)
output_data = []
for idx, (qa_id, title, body, answer) in test_data:
    output = network(title, body, answer).tolist()
    output.insert(0, qa_id)
    output_data.append(output)
    print(f'Testing... {idx + 1}/{total}', end='\r')
print('Testing... Done!  ')

output_df = pd.DataFrame(output_data, columns=test_labels)
output_df.to_csv('submission.csv', index=False)
print('Testing output saved to submission.csv')